### Credit

This notebook is a modified version of one of the publicly provided training notebooks by [unsloth.ai](https://unsloth.ai/) -> [Github](https://github.com/unslothai/unsloth).

All credit goes to Daniel Han Chen and the team at unsloth.ai.

### Goal

- Perform 16bit LoRA finetuning of llama3-8b-instruct on our custom finetuning dataset
- Save .gguf versions of the model and upload it to the huggingface hub
    - f16 (no quant) and q8_0, q

- Hopefully obtain superior performance on our extraction task compared to the original model (performance evaluation is not part of this notebook)

In [1]:
# mount google drive and define paths (do this in first cell because colab makes us click to confirm in a context window...)
from google.colab import drive
drive.mount('/content/drive')
dir_path = "/content/drive/MyDrive/Data_Science_Studies/thesis_colab_notebooks"
data_path = f"{dir_path}/data"
output_path = f"{dir_path}/outputs"

# get hf access token (to download gated models/data and be able to upload models)
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')

Mounted at /content/drive


In [2]:
%%capture
# set up environment
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.26" trl peft accelerate bitsandbytes

In [3]:
from unsloth import FastLanguageModel
import torch
################################################################################
model_name = "meta-llama/Meta-Llama-3-8B-Instruct" # original meta link or f16 llama3 version in my hf repo
max_seq_length = 4*1024
dtype = None  # auto detection
load_in_4bit = False 
################################################################################

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = hf_token, 
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.5
   \\   /|    GPU: NVIDIA L4. Max memory: 22.168 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Unsloth: meta-llama/Meta-Llama-3-8B-Instruct has no tokenizer.model file.
Just informing you about this - this is not a critical error.


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
meta-llama/Meta-Llama-3-8B-Instruct does not have a padding token! Will use pad_token = <|reserved_special_token_250|>.


In [4]:
!nvidia-smi

Tue May 14 01:32:37 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   62C    P0              31W /  72W |  15653MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

Adding LoRA adapters to the model

In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 64, 
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, 
    bias = "none",    
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context # note: gradient checkpointing is a trade-off: more compute time for lower memory footprint
    random_state = 42,
    use_rslora = False,  
    loftq_config = None, 
)

Unsloth 2024.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


### Dataset & Prompts


In [6]:
import pandas as pd
from datasets import Dataset



# load data
# biased dataset? ADJUST!
ft_data_path = f"{data_path}/FT_biased0.30empty_transcript_chunks_nvids45968_chunksize2048_overlap50_tokMistral_with_metadata_for_prompt_with_labels.csv"
data = pd.read_csv(ft_data_path, sep=";")
data = Dataset.from_pandas(data)


# import prompt formatting function from custom module
import sys
sys.path.append('/content/drive/MyDrive/Data_Science_Studies/thesis_colab_notebooks/modules/')
from LLM_utils import format_prompt

# add formatted prompt (in this case including completions) to dataset
prompt_format_kwargs={
    "prompt_format": "llama3",
    "include_bos": True,
    "include_answer_tease": True,
    "include_label": True,
    "include_eos": True
    }
data = data.map(format_prompt, fn_kwargs=prompt_format_kwargs)

# since we want to train on completions only (i.e. compute loss only on the part we want the model to generate, not the whole prompt),
# we need to use the DataCollatorForCompletionOnlyLM and provide it with a response template (indicates the start of the completion part)
# in our case we can use the answer tease (if it is in the prompt) as the response template.
# IMPORTANT: it must look exactly like it does in LLM_utils.format_prompt(). Check trailing space, etc.!
from trl import DataCollatorForCompletionOnlyLM
answer_tease = "Sure, here's the json list with extracted trade recommendations:\n\n" # we can be 100% sure that the answer tease does not appear in the transcripts (no punctuation)
response_template = f"{answer_tease}"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)


Map:   0%|          | 0/213 [00:00<?, ? examples/s]

In [7]:
data

Dataset({
    features: ['video_id', 'chunk_number', 'chunk_text', 'uploader_id', 'yt_video_type', 'title', 'first_three_tags', 'label', 'prompt'],
    num_rows: 213
})

In [8]:
data[1]['prompt']

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a smart and efficient assistant specialized at extracting relevant information from text and replying in json format. You always follow the user\'s instructions carefully.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nThe triple-quoted text below is part of a youtube video transcript by channel @FinancialEducation with the title \'What Makes A Great Dividend Stock For Dividend Investing?\'. The top tags for the video are: \'dividends, dividend stocks, dividend investing\'. Read the transcript carefully in order to perform the asset name extraction task specified below the transcript.\n\n"""balance sheet and I can promise you a balance sheet can get a company into so much trouble if a company is loaded up on debts okay they\'re loaded up on debts and they don\'t have a lot of cash and investments around then you\'re looking at a company that\'s probably gonna have to make mass interest payments on that debt o

### Training Setup

In [9]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model=model,
    train_dataset=data,
    formatting_func=None,
    data_collator=collator, # DataCollatorForCompletionOnlyLM
    dataset_text_field="prompt",
    packing=False,
    max_seq_length=4*1024,
    args = TrainingArguments(
        num_train_epochs=1,
        per_device_train_batch_size = 1, # might have to use 1 for our prompt size?
        gradient_accumulation_steps = 1, # keep at 1 for our small dataset
        warmup_steps = 2, # probably zero or very low (> 0 might be better for adamw optimizer?)
        #max_steps = -1, # !!! careful with this if whole epoch should be completed!
        learning_rate = 1e-4, # what's best? probably rather small since we will use guided generation, we only want to nudge the model a little bit (tried 5e-5, 2e-4, )
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit", # adamw_8bit should be fine (gpu-memory efficient)
        weight_decay = 0.005, # best choice?
        lr_scheduler_type = "linear",
        seed = 42,
        output_dir = "outputs", # Do NOT set on gdrive!
    )
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/213 [00:00<?, ? examples/s]

In [10]:
# show pre-training memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA L4. Max memory = 22.168 GB.
15.709 GB of memory reserved.


In [11]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 213 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 1
\        /    Total batch size = 1 | Total steps = 213
 "-____-"     Number of trainable parameters = 167,772,160


Step,Training Loss
1,0.829800
2,0.194300
3,0.425700
4,0.242200
5,0.826600
6,0.037200
7,0.250100
8,0.037600
9,0.354200
10,0.034200


In [12]:
# show post-training memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

453.9053 seconds used for training.
7.57 minutes used for training.
Peak reserved memory = 18.473 GB.
Peak reserved memory for training = 2.764 GB.
Peak reserved memory % of max memory = 83.332 %.
Peak reserved memory for training % of max memory = 12.468 %.


### Quantized GGUF Upload

In [ ]:
########################################################
model_name = "llama3_8b_instruct_ft"
version_indicator = "v4"
########################################################
# save multiple variants
if True:
    for quantization_method in ["f16", "q8_0"]:
        full_model_name = f"{model_name}_{version_indicator}_{quantization_method}"
        model.push_to_hub_gguf(f"JanJacobsen/{full_model_name}", tokenizer, quantization_method=quantization_method, token=hf_token)


In [ ]:
if False:
    quantization_method = "q6_k" # "f16", "q8_0", "q6_k", "q4_k_m", ...
    full_model_name = f"{model_name}_{version_indicator}_{quantization_method}"
    # save locally
    if False: model.save_pretrained_gguf(full_model_name, tokenizer, quantization_method=quantization_method)
    # upload to huggingface
    if True: model.push_to_hub_gguf(f"JanJacobsen/{full_model_name}", tokenizer, quantization_method=quantization_method, token=hf_token)